# Simple Multi-Agent Chat System By Sahar Ejaz

In [ ]:
"""
Components from my coding project:
- Coordinator Agent (Manager)
- Research Agent
- Analysis Agent
- Memory Agent
- Enhanced Memory System

I have created 5 scenarios to comeplete this assignment, so just run this code to see all 5 test scenarios!
"""

import datetime
import json

# ============================================================================
# Enhanced Memory System
# ============================================================================

class EnhancedMemorySystem:
    """Stores conversations, knowledge, and agent states"""

    def __init__(self):
        # Conversation Memory: Full history with timestamps
        self.conversation_memory = []

        # Knowledge Base: Facts with provenance
        self.knowledge_base = {
            "neural networks": {"content": "Main types: feedforward, CNN (images), RNN (sequences), transformers (language)", "confidence": 0.9},
            "machine learning optimization": {"content": "Techniques: Gradient Descent, Adam (best for most cases), SGD. Adam is most effective", "confidence": 0.8},
            "transformer architectures": {"content": "Models like BERT, GPT. Great for language but need lots of computing power", "confidence": 0.85},
            "reinforcement learning": {"content": "Learning through rewards. Methods: DQN, Policy Gradient. Challenges: sample efficiency, reward design", "confidence": 0.8}
        }

        # Agent State Memory: What each agent did
        self.agent_states = {"research": [], "analysis": [], "memory": []}

    def store_conversation(self, query, response, agents_used):
        """Store full conversation with metadata"""
        record = {
            "timestamp": datetime.datetime.now().isoformat(),
            "query": query,
            "response": response,
            "agents_used": agents_used,
            "id": len(self.conversation_memory)
        }
        self.conversation_memory.append(record)

    def search_conversations(self, keywords):
        """Search conversations by keywords"""
        results = []
        for conv in self.conversation_memory:
            text = (conv["query"] + " " + conv["response"]).lower()
            if any(word.lower() in text for word in keywords):
                results.append(conv)
        return results

    def update_agent_state(self, agent, action, result):
        """Track agent actions"""
        self.agent_states[agent].append({
            "timestamp": datetime.datetime.now().isoformat(),
            "action": action,
            "result": result
        })

# ============================================================================
# Research Agent
# ============================================================================

class ResearchAgent:
    """Simulates information retrieval using pre-loaded knowledge base"""

    def __init__(self, memory_system):
        self.memory = memory_system

    def search_information(self, query):
        """Search knowledge base for information"""
        print(f"[RESEARCH AGENT] Searching: {query}")

        results = []
        query_lower = query.lower()

        # Search knowledge base
        for topic, data in self.memory.knowledge_base.items():
            if any(word in query_lower for word in topic.split()) or any(word in data["content"].lower() for word in query_lower.split()[:3]):
                results.append({
                    "topic": topic,
                    "content": data["content"],
                    "confidence": data["confidence"]
                })

        # Update agent state
        self.memory.update_agent_state("research", f"searched: {query}", {"found": len(results)})

        print(f"[RESEARCH AGENT] Found {len(results)} results")
        return results

# ============================================================================
# Analysis Agent
# ============================================================================

class AnalysisAgent:
    """Performs comparisons, reasoning, and calculations"""

    def __init__(self, memory_system):
        self.memory = memory_system

    def analyze_data(self, data):
        """Compare and analyze information"""
        print(f"[ANALYSIS AGENT] Analyzing {len(data)} items")

        if not data:
            return {"error": "No data to analyze"}

        # Simple analysis with confidence scoring
        analysis = []
        for item in data:
            score = item.get("confidence", 0.5)
            # Boost score for positive keywords
            if any(word in item["content"].lower() for word in ["best", "effective", "good"]):
                score += 0.1

            analysis.append({
                "topic": item["topic"],
                "content": item["content"],
                "effectiveness_score": score,
                "recommendation": "High" if score > 0.8 else "Medium" if score > 0.6 else "Low"
            })

        # Sort by effectiveness
        analysis.sort(key=lambda x: x["effectiveness_score"], reverse=True)

        result = {
            "analysis": analysis,
            "best_option": analysis[0]["topic"] if analysis else None,
            "confidence": sum(item["effectiveness_score"] for item in analysis) / len(analysis)
        }

        # Update agent state
        self.memory.update_agent_state("analysis", "analyzed data", result)

        print(f"[ANALYSIS AGENT] Best option: {result['best_option']}")
        return result

# ============================================================================
# Memory Agent
# ============================================================================

class MemoryAgent:
    """Manages long-term storage, retrieval, and context updates"""

    def __init__(self, memory_system):
        self.memory = memory_system

    def search_by_keywords(self, keywords):
        """Search conversations by keywords"""
        print(f"[MEMORY AGENT] Searching for: {keywords}")

        results = self.memory.search_conversations(keywords)

        # Update agent state
        self.memory.update_agent_state("memory", f"searched: {keywords}", {"found": len(results)})

        print(f"[MEMORY AGENT] Found {len(results)} conversations")
        return results

    def store_finding(self, topic, content, source_agent):
        """Store new knowledge with provenance"""
        self.memory.knowledge_base[topic] = {
            "content": content,
            "source": source_agent,
            "timestamp": datetime.datetime.now().isoformat(),
            "confidence": 0.7
        }

# ============================================================================
# Coordinator Agent (Manager)
# ============================================================================

class CoordinatorAgent:
    """Orchestrates three specialized worker agents"""

    def __init__(self):
        # Initialize Enhanced Memory System
        self.memory_system = EnhancedMemorySystem()

        # Initialize worker agents
        self.research_agent = ResearchAgent(self.memory_system)
        self.analysis_agent = AnalysisAgent(self.memory_system)
        self.memory_agent = MemoryAgent(self.memory_system)

        print("[COORDINATOR] Multi-agent system initialized")

    def process_query(self, user_query):
        """Main coordination method"""
        print(f"\n{'='*50}")
        print(f"[COORDINATOR] Processing: {user_query}")
        print('='*50)

        agents_used = []

        # Analyze complexity and determine agent sequence
        if self._is_memory_query(user_query):
            # Memory query
            response = self._handle_memory_query(user_query, agents_used)
        elif self._needs_analysis(user_query):
            # Complex query: Research → Analysis
            response = self._handle_complex_query(user_query, agents_used)
        else:
            # Simple query: Research only
            response = self._handle_simple_query(user_query, agents_used)

        # Store interaction
        self.memory_system.store_conversation(user_query, response, agents_used)

        print(f"[COORDINATOR] Used agents: {', '.join(agents_used)}")
        return response

    def _is_memory_query(self, query):
        """Check if query asks about previous conversations"""
        memory_words = ["earlier", "before", "discussed", "talked about", "learned"]
        return any(word in query.lower() for word in memory_words)

    def _needs_analysis(self, query):
        """Check if query needs analysis"""
        analysis_words = ["compare", "analyze", "which is better", "effectiveness", "recommend"]
        return any(word in query.lower() for word in analysis_words)

    def _handle_memory_query(self, query, agents_used):
        """Handle memory-related queries"""
        agents_used.append("MemoryAgent")

        # Extract search terms
        words = query.lower().split()
        search_words = [w for w in words if len(w) > 3 and w not in ["what", "did", "earlier", "about"]]

        results = self.memory_agent.search_by_keywords(search_words)

        if results:
            response = f"Found {len(results)} previous conversation(s):\n\n"
            for i, conv in enumerate(results[-2:], 1):  # Show last 2
                response += f"{i}. {conv['query']}\n   Answer: {conv['response'][:100]}...\n\n"
        else:
            response = "No previous conversations found about that topic."

        return response

    def _handle_simple_query(self, query, agents_used):
        """Handle simple research queries"""
        agents_used.append("ResearchAgent")

        research_results = self.research_agent.search_information(query)

        if research_results:
            response = "Here's what I found:\n\n"
            for item in research_results:
                response += f"**{item['topic'].title()}**: {item['content']}\n\n"
        else:
            response = "No information found for that query."

        return response

    def _handle_complex_query(self, query, agents_used):
        """Handle queries needing research + analysis"""
        agents_used.extend(["ResearchAgent", "AnalysisAgent"])

        # Step 1: Research
        research_results = self.research_agent.search_information(query)

        if not research_results:
            return "Insufficient information for analysis."

        # Step 2: Analysis
        analysis_results = self.analysis_agent.analyze_data(research_results)

        # Step 3: Synthesize
        response = "Research and Analysis Results:\n\n"
        response += f"**Best Recommendation**: {analysis_results['best_option']}\n"
        response += f"**Confidence**: {analysis_results['confidence']:.1%}\n\n"
        response += "**Analysis Details**:\n"

        for item in analysis_results['analysis'][:3]:  # Top 3
            response += f"- {item['topic']}: {item['recommendation']} effectiveness\n"

        return response

# ============================================================================
# Test System - Run 5 Required Scenarios
# ============================================================================

def run_sample_scenarios():
    """Run all 5 required test scenarios"""

    print("Multi-Agent System - Running Sample Test Scenarios")

    coordinator = CoordinatorAgent()

    # 5 Required test scenarios
    scenarios = [
        "What are the main types of neural networks?",  # Simple
        "Research transformer architectures, analyze their computational efficiency, and summarize key trade-offs.",  # Complex
        "What did we discuss about neural networks earlier?",  # Memory
        "Find recent papers on reinforcement learning, analyze their methodologies, and identify common challenges.",  # Multi-step
        "Compare two machine-learning approaches and recommend which is better for our use case."  # Collaborative
    ]

    for i, query in enumerate(scenarios, 1):
        print(f"\n{'='*60}")
        print(f"SCENARIO {i}: {['Simple Query', 'Complex Query', 'Memory Test', 'Multi-step', 'Collaborative'][i-1]}")
        print(f"{'='*60}")

        response = coordinator.process_query(query)

        print(f"\n[RESPONSE]:\n{response}")
        print(f"\n Scenario {i} completed")

    print(f"\n All {len(scenarios)} test scenarios completed successfully!")

# ============================================================================
# Main Execution
# ============================================================================

if __name__ == "__main__":
    run_sample_scenarios()

Multi-Agent System - Running Sample Test Scenarios
[COORDINATOR] Multi-agent system initialized

SCENARIO 1: Simple Query

[COORDINATOR] Processing: What are the main types of neural networks?
[RESEARCH AGENT] Searching: What are the main types of neural networks?
[RESEARCH AGENT] Found 1 results
[COORDINATOR] Used agents: ResearchAgent

[RESPONSE]:
Here's what I found:

**Neural Networks**: Main types: feedforward, CNN (images), RNN (sequences), transformers (language)



 Scenario 1 completed

SCENARIO 2: Complex Query

[COORDINATOR] Processing: Research transformer architectures, analyze their computational efficiency, and summarize key trade-offs.
[RESEARCH AGENT] Searching: Research transformer architectures, analyze their computational efficiency, and summarize key trade-offs.
[RESEARCH AGENT] Found 2 results
[ANALYSIS AGENT] Analyzing 2 items
[ANALYSIS AGENT] Best option: neural networks
[COORDINATOR] Used agents: ResearchAgent, AnalysisAgent

[RESPONSE]:
Research and Analysis R